x_train_no_blanks is just X_train with all the columns that Maruhan dropped by inspection in the beginning and with blanks filled in with -3.

In [ ]:
y_no_zeroes = y
for column in y_no_zeroes.columns:
    y_no_zeroes[column] = y_no_zeroes[column].replace(-1, 0)

print(y_no_zeroes.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import xgboost as xgb

# The X_train_no_blanks had l_data before
X_train_xg, X_test_xg, y_train_xg, y_test_xg = train_test_split(X_train_no_blanks, y_no_zeroes, random_state=20851822, test_size=0.2)

In [ ]:
xgboost_model = xgb.XGBClassifier(objective="multi:softmax", subsample=0.5, random_state=20851822)
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=20851822)

interaction_vals = [42, 46, 50, 54, 58, 62]# old values[50, 54, 58, 62, 66, 70]  best value #[62]
shrinkage_vals =  [1, 0.1, 0.01] #[0.1]
param_grid = dict(max_leaves=interaction_vals, learning_rate=shrinkage_vals)

grid_search = GridSearchCV(xgboost_model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=stratkfold)

grid_search.fit(X_train_xg, y_train_xg.values.ravel())

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

pred_new = best_model.predict_proba(X_test_xg)

multiclass_loss = loss(pred_new, y_test_xg.values.ravel())
print("Multiclass Log Loss:", multiclass_loss)

print("Best Hyperparameters:", best_params)